In [76]:
%run ItemModel.ipynb
%run APIService.ipynb

In [44]:
import inspect, re
import pandas as pd
import csv

# Class GetData

In [78]:
class GetData(apiServiceDelegate):
    newest = 0
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
    APIService = APIService(newest, keyword = "thoi trang nam", categoryids = "1899")
    
    i = 1
    def __init__(self, newest):
        self.newest = newest
        self.APIService.delegate = self
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def getData(self):
        while True:
            try:
                self.APIService.setNewest(self.newest)
                check = self.APIService.get()
                if check == False: break
                
            except Exception as e:
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        self.all_items.clear()
        for item in self.items_json:
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.historical_sold = item['historical_sold']
            itemModel.sold = item['sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            for tier_variation in item['tier_variations']:
                itemModel.options = len(tier_variation['options'])
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.flash_sale = item['flash_sale']
            itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            itemModel.video_info_list = len(item['video_info_list'])
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'name\tshop_location\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\trating_count\t'
                + f'flash_sale\tupcoming_flash_sale\tprice_min\tprice_min_before_discoun\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'sold\thistorical_sold\n')

        for item in self.all_items:
                file.write(f'{item.name}\t{item.shop_location}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.rating_count}\t'
                + f'{item.flash_sale}\t{item.upcoming_flash_sale}\t{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.sold}\t{item.historical_sold}\n')
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata))
        print(f'get! {self.i}')
        
        if json_pydata['items'] != None:
            self.items_json.extend(json_pydata['items'])
        else:
            self.parseData()
            print('get All Data!')

        self.newest += 50
        self.i += 1
        time.sleep(10)
        return True
        pass
    def getDataError(self, error):
        print(f'Error when request: {error}')
        return False
        pass
    
    pass

# Get Data

In [79]:
getData = GetData(0)

In [80]:
getData.getData()


Error when request: Expecting value: line 1 column 1 (char 0)


In [81]:
getData.parseData()

In [14]:
getData.APIService.json_pydata

{'show_disclaimer': None,
 'query_rewrite': None,
 'adjust': None,
 'version': '453b4ce4955c7c8068c54bc30a274f21',
 'algorithm': None,
 'need_next_search': None,
 'total_count': None,
 'error': None,
 'total_ads_count': None,
 'disclaimer_infos': None,
 'nomore': None,
 'price_adjust': None,
 'json_data': None,
 'suggestion_algorithm': None,
 'items': None,
 'reserved_keyword': None,
 'hint_keywords': None}

In [20]:
getData.APIService.headers


{}

In [16]:
# getData.APIService.url

In [72]:
# Test ParseData
print(getData.all_items[0].name)
print(getData.all_items[50].name)
print(getData.all_items[1].name)
print(getData.all_items[51].name)

Áo Thun Trơn Nam - Áo phông nam tay ngắn 100% cotton Trắng Đen dáng ôm vừa vặn AT01
Áo thun nam tay ngắn không cổ chất cotton tici phối hình - CZ23
Áo thun tay lở form rộng thêu hình vuông -  Unisex nam nữ đều mặc được
Áo thun nam nữ tay lỡ YINXX, áo phông form rộng ATL193


In [73]:
getData.saveFile("ShoppeItem_thoiTrangNam_Quan.tsv")

In [74]:
# Test saveFile
items_df = pd.read_csv('ShoppeItem_thoiTrangNam_Quan.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df


,name,shop_location,shopee_verified,discount,is_adult,options,show_official_shop_label_in_title,rating_star,rating_count,flash_sale,...,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,sold,historical_sold
0,Áo Thun Trơn Nam - Áo phông nam tay ngắn 100% cotton Trắng Đen dáng ôm vừa vặn AT01,TP. Hồ Chí Minh,True,49%,False,5,False,4.951220,"[164, 0, 0, 2, 4, 158]",None,...,None,104,9780,164,True,False,9,0,224,490
1,Áo thun tay lở form rộng thêu hình vuông - Unisex nam nữ đều mặc được,TP. Hồ Chí Minh,True,45%,False,4,False,4.897638,"[127, 0, 1, 2, 6, 118]",None,...,None,345,17771,127,False,False,9,0,235,292
2,Áo thun nam cổ tròn cao cấp RIVER manZONE ZONE12,TP. Hồ Chí Minh,False,27%,False,4,False,0.000000,"[0, 0, 0, 0, 0, 0]",None,...,None,0,129,0,False,False,9,0,0,0
3,FreeShip Đơn 50K ⚡️Siêu Phẩm⚡️ Áo thun nam nữ unisex tay lỡ in hình ông trùm mèo đẹp vải dày mịn,Đồng Nai,True,None,False,11,False,4.789474,"[228, 6, 2, 6, 6, 208]",None,...,None,12235,6434,228,False,False,6,0,145,644
4,ÁO THUN TRƠN BA LỖ NAM ĐỦ MÀU - SNN000,TP. Hồ Chí Minh,True,34%,False,7,False,4.705357,"[112, 2, 1, 6, 10, 93]",None,...,None,364,2408,112,False,True,6,0,139,536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Áo thun thời trang nam - Yêu âm nhạc - Love music - Guitar - Ghita cực cool dễ thương Unisex -Log69 - Lilymall.net 095,TP. Hồ Chí Minh,False,40%,False,8,False,0.000000,"[0, 0, 0, 0, 0, 0]",None,...,None,0,0,0,False,False,1,0,0,0
996,Áo Death Note🔥HOT🔥Áo in hình Manga đẹp giá rẻ thời trang nam /uy tín chất lượng,Hà Nội,False,None,False,5,False,0.000000,"[0, 0, 0, 0, 0, 0]",None,...,None,0,0,0,False,False,1,0,0,0
997,Áo thun cặp big size - Áo Thun Nam Thời Trang Nam - Áo Cho Người Béo,Bình Định,True,38%,False,11,False,5.000000,"[3, 0, 0, 0, 0, 3]",None,...,None,0,35,3,False,False,2,0,0,4
998,"Thời trang nam, áo thun ngắn tay không cổ ANT002",TP. Hồ Chí Minh,False,12%,False,4,False,0.000000,"[0, 0, 0, 0, 0, 0]",None,...,None,370,11,0,False,False,7,0,0,1


In [ ]:
del items_df